# 強化学習

In [12]:
from __future__ import division
import numpy as np
import random
import math
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn

## 多腕バンディット問題

### greedy algorithm

In [6]:
#引数 -> dict {方策名(番号?), [プレイした回数,報酬の和]}
#return -> 選択した方策
def greedy(status_dict, n):
    dict_keys = status_dict.keys()
    play_num_ary = np.array([status_dict[key][0] for key in dict_keys])
    if np.min(play_num_ary) < n:
        return play_num_ary[np.argmin(play_num_ary)]
    else:
        mu_lst = np.array([status_dict[key][1] / status_dict[key][0] for key in dict_keys])
        return dict_keys[np.argmax(mu_lst)]

### ε-greedy algorithm

In [59]:
#引数 -> dict {方策名(番号?), [プレイした回数,報酬の和]}
#return -> 選択した方策
def eps_greedy(status_dict, eps):
    dict_keys = status_dict.keys()
    play_num_ary = np.array([status_dict[key][0] for key in dict_keys])
    if np.min(play_num_ary) == 0:
        return dict_keys[np.argmin(play_num_ary)]
    if np.random.rand() <= eps:
        return random.choice(np.array(dict_keys))
    else:
        mu_lst = np.array([status_dict[key][1] / status_dict[key][0] for key in dict_keys])
        return dict_keys[np.argmax(mu_lst)]

### 楽観的初期値法(optimistic initial values)

In [6]:
#引数 -> dict {方策名(番号?), [プレイした回数,報酬の和]}, r_sup : 各腕からの報酬の上界, K : r_supが観測された(見積もり)回数
#return -> 選択した方策
def optimistic_init_val(status_dict, r_sup, K=1):
    dict_keys = status_dict.keys()
    print dict_keys
    mu_lst = np.array([(status_dict[key][1] + K * r_sup) / (status_dict[key][0] + K) for key in dict_keys])
    print mu_lst
    return dict_keys[np.argmax(mu_lst)]

### UCB1 (Upper Confidence Bound) algorithm

In [13]:
#引数 -> dict {方策名(番号?), [プレイした回数,報酬の和]}, R : はらいもどしがくん最大値と最小値の差
#return -> 選択した方策
def UBC1(status_dict, R):
    dict_keys = status_dict.keys()
    play_num_ary = np.array([status_dict[key][0] for key in dict_keys])
    if np.min(play_num_ary) == 0:
        return dict_keys[np.argmin(play_num_ary)]
    mu_lst = np.array([status_dict[key][1] / status_dict[key][0] for key in dict_keys])
    U_bunshi = 2 * math.log(np.sum(play_num_ary))
    U_lst = np.array([R * math.sqrt(U_bunshi / status_dict[key][0]) for key in dict_keys])
    return dict_keys[np.argmax(mu_lst+U_lst)]  

In [ ]:
#結果のプロットはmatplotlibのstackplotでできそう